In [2]:
! gdown 1yEjicXasMGFaALQobQWgsSxQgLEtxruC
! unzip /content/ThaiDepression.zip

Downloading...
From: https://drive.google.com/uc?id=1yEjicXasMGFaALQobQWgsSxQgLEtxruC
To: /content/ThaiDepression.zip
100% 41.1k/41.1k [00:00<00:00, 52.6MB/s]
Archive:  /content/ThaiDepression.zip
  inflating: ThaiDepression/AHB_01.csv  
  inflating: ThaiDepression/AHB_02.csv  
  inflating: ThaiDepression/AHB_03.csv  
  inflating: ThaiDepression/AHB_04.csv  
  inflating: ThaiDepression/AHB_05.csv  
  inflating: ThaiDepression/AHB_06.csv  
  inflating: ThaiDepression/AHB_07.csv  
  inflating: ThaiDepression/AHB_08.csv  
  inflating: ThaiDepression/AHB_09.csv  
  inflating: ThaiDepression/AHB_10.csv  
  inflating: ThaiDepression/AHB_11.csv  
  inflating: ThaiDepression/AHB_12.csv  
  inflating: ThaiDepression/AHB_13.csv  
  inflating: ThaiDepression/All_AHB.csv  


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14762029797012465709
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 40263417856
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8509322903411665619
physical_device_desc: "device: 0, name: A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0"
xla_global_id: 416903419
]


In [4]:
import pandas as pd
import numpy as np
import os
from numpy import array
from math import sqrt
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
import json

In [5]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))

In [6]:
df_list

[         Chiang_Rai    Nan  Phayao  Phrae  Chiang_Mai  Mae_Hong_Son  Lampang  \
 2016-10       10383   5315    4266   4057       15408          2002     9011   
 2016-11       10383   5315    4265   4057       15406          2002     9011   
 2016-12       10382   5315    4265   4057       15404          2002     9011   
 2017-01       10382   5655    4265   4057       15393          2002     9011   
 2017-02       10382   5655    4264   4057       15392          2002     9011   
 ...             ...    ...     ...    ...         ...           ...      ...   
 2022-03       25223   9711   10788   7988       47292          6924    17444   
 2022-04       25456   9781   10932   8095       48020          7004    17606   
 2022-05       25619   9833   10977   8122       48155          7026    17703   
 2022-06       25836   9884   11069   8161       48616          7090    17783   
 2022-07       26174  10019   11163   8250       49096          7169    17951   
 
          Lamphun   AHB_1 

In [7]:
# split a univariate sequence into samples
def uni_split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


    # split a multivariate sequence into samples
def multi_split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

In [12]:
%matplotlib inline
# define scope of configs
AHB_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
n_filters = [16, 32, 64, 128]
n_kernels = [2, 3]
n_layer = [2]
n_batch = [2, 4, 8, 16, 32, 64, 128]
n_steps = [4]
patience_list = [3, 5, 7, 10]
mul_uni = ['uni','mul']

count = 1
performance_json = []
for g in AHB_list:
    for h in patience_list:
        for a in n_steps:
            for b in n_filters:
                for c in n_kernels:
                    for d in n_layer:
                        for e in n_batch:
                            for f in mul_uni:

                                print('PROGRESS: [ ' + str(count) + ' / ' + str(len(AHB_list)*len(n_steps)*len(n_filters)*len(n_kernels)*len(n_layer)*len(n_batch)*len(patience_list)*len(mul_uni)) + ' ]')
                                
                                if f == 'uni':
                                    if g == 13:
                                        raw_seq = list(df_list[g]['Country_Level'])
                                        ahb = 'Country_Level'
                                    else:    
                                        raw_seq = list(df_list[g]['AHB_'+str(g+1)])
                                        ahb = 'AHB_'+str(g+1)
                                    n_features = 1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq).reshape(len(raw_seq),-1))
                                    X, y = uni_split_sequence(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
                                if f == 'mul':
                                    if g == 13:
                                        raw_seq = list(df_list[13].drop(df_list[13].columns[:77], axis=1).values)
                                        ahb = 'Country_Level'
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[13]['Country_Level']).reshape(len(df_list[13]['Country_Level']),-1))
                                    else:    
                                        raw_seq = list(df_list[g].drop(['Country_Level'], axis=1).values)
                                        ahb = 'AHB_'+str(g+1)
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[g]['AHB_'+str(g+1)]).reshape(len(df_list[13]['AHB_'+str(g+1)]),-1))
                                    n_features = len(raw_seq[0])-1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq))
                                    X, y = multi_split_sequences(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

                                callback = EarlyStopping(monitor='loss', patience=h)
                                model = Sequential()
                                model.add(Conv1D(b, c, activation='relu', input_shape=(a,n_features)))
                                model.add(MaxPooling1D())
                                if d == 2:
                                    model.add(Conv1D(int(b*2), int(c/2), activation='relu'))  
                                model.add(Flatten())
                                model.add(Dense(50, activation='relu'))
                                model.add(Dense(1))
                                model.compile(optimizer='adam', loss='mse')
                                history = model.fit(X_train, y_train, epochs=1000, batch_size=e, verbose=0, callbacks=[callback])
                                last_epoch = len(history.history['loss']) 

                                yhat = model.predict(X_test, verbose=0)
                                mae = mean_absolute_error(y_test, yhat)
                                mse = mean_squared_error(y_test, yhat)
                                rmse = mean_squared_error(y_test, yhat, squared = False)
                                mape = mean_absolute_percentage_error(y_test, yhat)
                                
                                if f == 'uni':
                                    yhat_inverse = scaler.inverse_transform(yhat)
                                    y_test_inverse = scaler.inverse_transform(y_test)
                                if f == 'mul':
                                    yhat_inverse = scaler_y.inverse_transform(yhat)
                                    y_test_inverse = scaler_y.inverse_transform(y_test.reshape(len(y_test),-1))


                                mae_inverse = mean_absolute_error(y_test_inverse, yhat_inverse)
                                mse_inverse = mean_squared_error(y_test_inverse, yhat_inverse)
                                rmse_inverse = mean_squared_error(y_test_inverse, yhat_inverse, squared = False)
                                mape_inverse = mean_absolute_percentage_error(y_test_inverse, yhat_inverse)

                                performance_json.append({'model': '1D-CNN', 'AHB_number': ahb, 'Multi_Uni': f, \
                                                         'n_filters': b, 'n_layer': d, 'n_kernels': c, 'n_batch': e,\
                                                         'n_steps': a, 'last_epoch': last_epoch, 'patience_list': h, 'MAE_normalize': float(mae), \
                                                         'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                                         'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                

                                json_formatted_str = json.dumps(performance_json[-1], indent=2)
                                print(json_formatted_str)

                                count = count + 1

Streaming output truncated to the last 5000 lines.
  "MAPE": 0.07476192875089582
}
PROGRESS: [ 6035 / 6272 ]
{
  "model": "1D-CNN",
  "AHB_number": "Country_Level",
  "Multi_Uni": "uni",
  "n_filters": 128,
  "n_layer": 2,
  "n_kernels": 3,
  "n_batch": 2,
  "n_steps": 4,
  "last_epoch": 13,
  "patience_list": 5,
  "MAE_normalize": 0.008452020771173099,
  "MSE_normalize": 8.15489459568439e-05,
  "RMSE_normalize": 0.009030445501571,
  "MAPE_normalize": 0.008752718402960876,
  "MAE": 5434.607142857143,
  "MSE": 33715792.102678575,
  "RMSE": 5806.530125873676,
  "MAPE": 0.004424069652547119
}
PROGRESS: [ 6036 / 6272 ]
{
  "model": "1D-CNN",
  "AHB_number": "Country_Level",
  "Multi_Uni": "mul",
  "n_filters": 128,
  "n_layer": 2,
  "n_kernels": 3,
  "n_batch": 2,
  "n_steps": 4,
  "last_epoch": 14,
  "patience_list": 5,
  "MAE_normalize": 0.010648423886288885,
  "MSE_normalize": 0.00011653272553808606,
  "RMSE_normalize": 0.010795032447291951,
  "MAPE_normalize": 0.011156084844888548,
  "

In [13]:
json.dump(performance_json, open("all_performance_1D_CNN_4nsteps_2.json","w"))
df_json = pd.read_json('/content/all_performance_1D_CNN_4nsteps_2.json')
df_json.to_csv('/content/all_performance_1D_CNN_4nsteps_2.csv', index=False)
#! cp /content/all_performance_1D_CNN_4nsteps_2.json /content/drive/MyDrive/Depression
! cp /content/all_performance_1D_CNN_4nsteps_2.csv /content/drive/MyDrive/Depression

In [14]:
df_json

,model,AHB_number,Multi_Uni,n_filters,n_layer,n_kernels,n_batch,n_steps,last_epoch,patience_list,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
0,1D-CNN,AHB_1,uni,16,2,2,2,4,13,3,0.031196,0.000987,0.031422,0.032412,2779.020089,7.835189e+06,2799.140725,0.019934
1,1D-CNN,AHB_1,mul,16,2,2,2,4,13,3,0.023961,0.000588,0.024259,0.024890,2134.453125,4.669916e+06,2160.998943,0.015309
2,1D-CNN,AHB_1,uni,16,2,2,4,4,18,3,0.030451,0.000945,0.030734,0.031621,2712.669643,7.495925e+06,2737.868665,0.019451
3,1D-CNN,AHB_1,mul,16,2,2,4,4,28,3,0.012085,0.000160,0.012648,0.012526,1076.584821,1.269474e+06,1126.709451,0.007711
4,1D-CNN,AHB_1,uni,16,2,2,8,4,53,3,0.018107,0.000339,0.018405,0.018795,1612.970982,2.688106e+06,1639.544435,0.011563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6267,1D-CNN,Country_Level,mul,128,2,3,32,4,170,10,0.002862,0.000010,0.003230,0.002970,1839.982143,4.313194e+06,2076.823028,0.001499
6268,1D-CNN,Country_Level,uni,128,2,3,64,4,65,10,0.009564,0.000098,0.009889,0.010033,6149.767857,4.043075e+07,6358.518201,0.005039
6269,1D-CNN,Country_Level,mul,128,2,3,64,4,779,10,0.003681,0.000018,0.004289,0.003789,2366.803571,7.607165e+06,2758.109024,0.001921
6270,1D-CNN,Country_Level,uni,128,2,3,128,4,1000,10,0.007338,0.000064,0.007992,0.007593,4718.357143,2.640959e+07,5139.026011,0.003840
